# START YOUR CODE HERE IN THIS NOTEBOOK, SIR/MS. :)

In [17]:
"""
PROJ-24
R2.2 - As a User, I want to choose a video file from the data folder 
through an appropriate UI component (e.g., dropdown menu) in a 
notebook code cell so that I can access the video easily.
"""
import os
from ipywidgets import interact, Dropdown

videoList = [] 

for x in os.listdir("./pipline/data/videos/sample_videos"): 
    videoList += [x]
# print(videoList)

videos = Dropdown(options = videoList) 

# using a list as a means to extract variable defined in python function.
selected_video = []
 
@interact(video = videos) 
def print_city(video): 
    print(video)
    # list shall only contain one element, that is the current selected video.
    selected_video.clear()
    selected_video.append(str(video))

interactive(children=(Dropdown(description='video', options=('sample_video_1.mp4', 'sample_video_2.mp4'), valu…

sample_video_1.mp4
